### 1. Running Total of Sales Year-to-Date
**Functional Specification:**
* Select `SalesOrderID`, `OrderDate`, and `TotalDue`.
* Use `SUM(TotalDue)` as a window function.
* Partition by the Year of the `OrderDate`.
* Order by `OrderDate` and `SalesOrderID`.
* Define the frame as `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` to calculate a cumulative running total.

In [ ]:
SELECT
        SalesOrderID,
        OrderDate,
        TotalDue,
            -- Calculate the Running Total (SUM of TotalDue)
        SUM(TotalDue) OVER (
            -- PARTITION BY the year of the OrderDate to reset the running total each year
            PARTITION BY YEAR(OrderDate)
            ORDER BY OrderDate,SalesOrderID
            --Accumulates rows all the way before and includes the current row
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) as RunningTotalYeartoDate
    FROM
        Sales.SalesOrderHeader
    ORDER BY
        YEAR(OrderDate),
        OrderDate,
        SalesOrderID;

### 2. Moving Average of Sales (3-Order Rolling)
**Functional Specification:**
* Select `SalesOrderID` and `TotalDue`.
* Use `AVG(TotalDue)`.
* Partition by `CustomerID` to calculate per customer.
* Order by `OrderDate`.
* Set frame to `ROWS BETWEEN 2 PRECEDING AND CURRENT ROW` to calculate the average of the current order and the previous two.

In [ ]:
SELECT
    CustomerID,
    SalesOrderID,
    OrderDate,
    TotalDue,
    -- Calculate the Moving Average (AVG of TotalDue)
    AVG(TotalDue) OVER (
        -- PARTITION BY customerID so it calculates each customer separately
        PARTITION BY CustomerID
        -- ORDER BY date
        ORDER BY OrderDate, SalesOrderID
        --Does the same thing as last time but doesnt go all the way, just gets 2 rows behind
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) as MovingAverageLast3Orders
FROM
    SAles.SalesOrderHeader
ORDER BY
    CustomerID,
    OrderDate,
    SalesOrderID;

### 3. Percent of Territory Total Sales
**Functional Specification:**
* Select `SalesOrderID`, `TerritoryID`, and `TotalDue`.
* Calculate the sum of `TotalDue` for the entire territory using `SUM(TotalDue) OVER (PARTITION BY TerritoryID)`.
* Divide the individual order `TotalDue` by this window sum to get the percentage contribution of that order to the territory's total.

In [ ]:
SELECT
    SalesOrderID,
    TerritoryID,
    TotalDue,
    -- Calculate the Total Sales for the entire territory (Partition Total)
    SUM(TotalDue) OVER (PARTITION BY TerritoryID) as TerritoryTotalSales,
    -- Divide individual TotalDue by the Partition Total to get the percentage
    (TotalDue/ SUM(TotalDue) OVER (PARTITION BY TerritoryID)) * 100 as PercentOfTerritoryTotal 
FROM
    Sales.SalesOrderHeader
ORDER BY
    TerritoryID,
    SalesOrderID;

### 4. Maximum Order Value per Customer Attached to Detail
**Functional Specification:**
* Select order details.
* Use `MAX(TotalDue)` as a window function.
* Partition by `CustomerID`.
* This allows us to see every order a customer made next to the largest order they ever made (for comparison).

In [ ]:
SELECT
    SOH.CustomerID,
    SOH.SalesOrderID,
    SOH.OrderDate,
    SOH.TotalDue as CurrentOrderValue,
    -- Calculate the maximum TotalDue for the specific customer
    MAX(SOH.TotalDue) OVER (PARTITION BY SOH.CustomerID) as MaximumOrderValue
FROM 
    Sales.SalesOrderHeader as SOH
ORDER BY 
    SOH.CustomerID,
    SOH.OrderDate DESC;

### 5. Cumulative Inventory Count by Location
**Functional Specification:**
* Query `Production.ProductInventory`.
* Partition by `LocationID`.
* Order by `ProductID` (or bin number).
* Use `SUM(Quantity)` with a frame `UNBOUNDED PRECEDING` to see how inventory accumulates as you walk down the shelf/bin list.

In [ ]:
SELECT
    LocationID,
    ProductID,
    Shelf,
    Bin,
    Quantity,
    -- Calculate the Cumulative Inventory Count (Running Total of Quantity)
    SUM(Quantity) OVER (
        -- PARTITION BY LocationID to contain to each location
        PARTITION BY LocationID
        -- ORDER BY ProductID and Bin
        ORDER BY ProductID, Bin
        --Uses this to find the entire cumulative inventory count from before rows
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) as CumulativeInventory
FROM
    Production.ProductInventory
ORDER BY
    LocationID,
    ProductID,
    Bin;

### 6. Count of Orders per Customer (Running)
**Functional Specification:**
* Select `CustomerID` and `SalesOrderID`.
* Use `COUNT(SalesOrderID)`.
* Partition by `CustomerID`.
* Order by `OrderDate`.
* This creates a sequence (1, 2, 3...) indicating this is the "Nth" order for that customer.

In [ ]:
SELECT
    CustomerID, 
    SalesOrderID,
    OrderDate,
    -- Calculate the running count of orders (1, 2, 3, etc.)
    COUNT(SalesOrderID) OVER (
        -- PARTITION BY CustomerID ensures the count resets for each customer
        PARTITION BY CustomerID
        -- ORDER BY date establishes the chronological sequence
        ORDER BY OrderDate, SalesOrderID
        
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS CustomerOrderSequence
FROM
    Sales.SalesOrderHeader
ORDER BY
    CustomerID,
    OrderDate,
    SalesOrderID;

### 7. Min Price in Category vs Product Price
**Functional Specification:**
* Join `Production.Product` and `Production.ProductSubcategory`.
* Use `MIN(ListPrice)` partitioned by `ProductSubcategoryID`.
* Select `Name`, `ListPrice`, and the calculated `MinCategoryPrice`.
* Filter where ListPrice > 0.

In [ ]:
SELECT
    PSC.Name AS ProductSubcategoryName, 
    P.Name AS ProductName,  
    P.ListPrice, 
    -- Calculate the minimum list price within the product's subcategory
    MIN(P.ListPrice) OVER (
        -- PARTITION BY the subcategory ID to define the comparison group
        PARTITION BY P.ProductSubcategoryID
    ) AS MinCategoryListPrice
FROM
    Production.product as P
    INNER JOIN Production.ProductSubcategory AS PSC ON P.ProductSubcategoryID = PSC.ProductSubcategoryID
    -- Filter out products that have no list price
WHERE
    P.ListPrice > 0 
ORDER BY 
    ProductSubcategoryName, 
    P.ListPrice; 

### 8. Average Order Line Item Total per Order
**Functional Specification:**
* Query `Sales.SalesOrderDetail`.
* Calculate `LineTotal`.
* Use `AVG(LineTotal)` partitioned by `SalesOrderID`.
* Return detailed line items next to the average line item value for that specific order.

In [ ]:
SELECT
    SalesOrderID,
    SalesOrderDetailID,
    ProductID,
    LineTotal,
    -- Calculate the average LineTotal for the entire order
    AVG(LineTotal) OVER (
        -- PARTITION BY SalesOrderID to define the group (the current order)
        PARTITION BY SalesOrderID
    ) as AverageLineItemTotalPerOrder
FROM
    Sales.SalesOrderDetail
ORDER BY 
    SalesOrderID,
    SalesOrderDetailID;

### 9. Total Sales per Year vs Global Total
**Functional Specification:**
* Select Year and `TotalDue`.
* Use `SUM(TotalDue)` partitioned by Year to get annual totals.
* Use `SUM(TotalDue) OVER ()` (empty window) to get the grand total of all sales ever.
* Use distinct to show one row per order (or group by year and sum first, then window).

In [ ]:
WITH YearlySales AS (  
    --Calculate the total sales for each year
    SELECT
        YEAR(OrderDate) AS SalesYear,
        SUM(TotalDue) AS AnnualTotalSales
    FROM
        Sales.SalesOrderHeader
    GROUP BY
        YEAR(OrderDate)  
)
SELECT
    SalesYear,
    AnnualTotalSales,
    --Calculate the Grand Total using an empty window function on the pre-aggregated data
    SUM(AnnualTotalSales) OVER () AS GlobalTotalSales,
    --Calculate the percentage contribution of the year to the global total
    (AnnualTotalSales / SUM(AnnualTotalSales) OVER ()) * 100 AS PercentOfGlobalSales 
FROM
    YearlySales
ORDER BY
    SalesYear;

In [ ]:
WITH YearlySales as (
    SELECT
        YEAR(OrderDate) as SalesYear,
        SUM(TotalDue) as AnnualTotalSales
    FROM
        Sales.SalesOrderHeader
    GROUP BY 
        YEAR(OrderDate)
)
SELECT
    SalesYear,
    AnnualTotalSales,
    SUM(AnnualTotalSales) OVER () as GlobalTotalSales,
    (AnnualTotalSales / SUM(AnnualTotalSales) OVER ()) * 100 as PercentofGlobalSales

FROM
    YearlySales
ORDER BY 
    SalesYear;

### 10. Count of Products per ProductLine
**Functional Specification:**
* Query `Production.Product`.
* Use `COUNT(ProductID)` partitioned by `ProductLine`.
* Return the Product Name and the count of how many products share its product line.

In [16]:
SELECT
    Name AS ProductName,
    ProductID, -- Included as the unique identifier
    -- Calculate the total number of products within the current ProductLine
    COUNT(ProductID) OVER (
        -- PARTITION BY ProductLine to group the rows and calculate the count independently for each line
        PARTITION BY ProductLine
    ) AS TotalProductsInLine
FROM
    Production.Product
ORDER BY
    ProductName;

(504 rows affected)

Total execution time: 00:00:00.016

ProductName,ProductID,TotalProductsInLine
Adjustable Race,1,226
All-Purpose Bike Stand,879,91
AWC Logo Cap,712,35
BB Ball Bearing,3,226
Bearing Ball,2,226
Bike Wash - Dissolver,877,35
Blade,316,226
Cable Lock,843,35
Chain,952,226
Chain Stays,324,226
